<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/conv_specification_example/tests/conv_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install onnx onnxruntime

In [1]:
import onnx
from onnx import TensorProto
from onnx.helper import (
    make_model, make_node, make_graph,
    make_tensor_value_info)
from onnx.checker import check_model

In [2]:
# @title Standard max pooling (1 channel)
import numpy as np
from onnx import *
import onnxruntime

# Create inputs
x = helper.make_tensor_value_info('x', TensorProto.DOUBLE, [1, 1, 8, 8])

# Create a node (MaxPool) with input/outputs
node_def = make_node(
    'MaxPool', # node name
    ['x'], # inputs
    ['y', 'Indices'], # outputs
    ceil_mode = 0,
    dilations=[1,1],
    kernel_shape=[3,3],
    pads=[1, 0, 1, 0],
    strides=[1, 1],
    auto_pad='NOTSET',
#    group=1, # Standard convolution
)

# Create the graph
graph_def = make_graph(
    [node_def],
    'test-maxpool',
    [x],
    [helper.make_tensor_value_info('y', TensorProto.DOUBLE, [1, 1, 8, 6]), 
     helper.make_tensor_value_info('Indices', TensorProto.INT64, [1, 1, 8, 6])],
)

onnx_model = make_model(graph_def)

# Let's freeze the opset.
del onnx_model.opset_import[:]
opset = onnx_model.opset_import.add()
opset.domain = ''
opset.version = 15
onnx_model.ir_version = 8

# Verify the model
check_model(onnx_model)

# Print a human-readable representation of the graph
print(onnx.helper.printable_graph(onnx_model.graph))

# Do inference
sess = onnxruntime.InferenceSession(onnx_model.SerializeToString(),
                        providers=["CPUExecutionProvider"])

# Initialize tensors
#x = numpy.ones((1, 1, 8, 8), dtype=numpy.float32)

x = np.random.normal(3, 2.5, size=(1, 1, 8, 8))

y = sess.run(None, {'x': x})[0]
Indices = sess.run(None, {'x': x})[1]

print("X shape:", x.shape)
print("X:", x)
print("Y shape:", y.shape)
print("Y:", y)
print("Indices shape:", Indices.shape)
print("Indices:", Indices)


graph test-maxpool (
  %x[DOUBLE, 1x1x8x8]
) {
  %y, %Indices = MaxPool[auto_pad = 'NOTSET', ceil_mode = 0, dilations = [1, 1], kernel_shape = [3, 3], pads = [1, 0, 1, 0], strides = [1, 1]](%x)
  return %y, %Indices
}
X shape: (1, 1, 8, 8)
X: [[[[ 4.43305943  1.05817534  4.28962538  4.28470842  3.00644965
     5.69287057  5.72557783  1.31154885]
   [ 5.43695168 -1.82703117  2.87699581  2.69635184  6.18242239
     7.48620983  1.11871504  1.60847976]
   [ 6.1525279   3.05861691  0.24934671  2.01243241  5.21568223
     6.78521868 -0.2558449   7.63857434]
   [ 1.04082415 -0.36311271  2.57890517  1.90215758  2.56666234
     2.53389661  6.59819875  4.78165774]
   [ 2.40125121  3.4556453   2.94774791  4.78412922  3.21581614
    -1.627449    1.59955765  4.47469832]
   [ 4.72100973  2.43828795  3.51116899 -2.88811044  6.53731096
     4.34773169  4.63030148  3.9411301 ]
   [ 3.00536547  6.61801766  3.65044952  1.56946725  4.29760772
     2.49783759  0.60523926  2.5508386 ]
   [ 3.20546648  5.906

/var/tmp/ipykernel_134949/1289850978.py:45: DeprecationWarning: Deprecated since 1.19. Consider using onnx.printer.to_text() instead.
  print(onnx.helper.printable_graph(onnx_model.graph))
